In [1]:
import os
import cv2
from deepface import DeepFace
import FaceNormalizationUtils as faceutils
import numpy as np
import FaceDetectors
import pygame


pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [7]:
import cv2
import threading
import time
from deepface import DeepFace

# Initialize normalization
normalizatorHS = faceutils.Normalization()

# Initialize face detector
FDet = FaceDetectors.FaceDetector()

# Fonts
font = cv2.FONT_HERSHEY_SIMPLEX

# Webcam connection
cap = cv2.VideoCapture(0)

# Set camera resolution
cap.set(3, 640)
cap.set(4, 480)

# Inicializar fondo predeterminado
wano = cv2.imread('media/wano.jpg')
wano = cv2.resize(wano, (int(cap.get(3)), int(cap.get(4))))

# Variable para almacenar el fondo actual
background = wano

# Variable para almacenar la raza detectada
race_detected = ""


canciones = {
    'asian': 'songs/china.mp3',
    'black': 'songs/africa.mp3',
    'white': 'songs/blanco.mp3',
    'indian': 'songs/india.mp3',
    'middle eastern': 'songs/moro.mp3',
    'latino hispanic': 'songs/macaco.mp3',
}


cancion_predeterminada = ''
prev_race = ''

# Inicializar Pygame
pygame.mixer.init()


# Función para cambiar el fondo cada 5 segundos
def cambiar_fondo():
    global background, race_detected, prev_race
    while True:
        
        # Lógica para cambiar el fondo según la raza detectada
        if race_detected != prev_race:
            prev_race = race_detected
            print('Cambiando fondo')
            if race_detected == "asian":
                background = cv2.imread('media/wano.jpg')
            elif race_detected == "black":
                background = cv2.imread('media/savannah.jpg')
            elif race_detected == "white":
                background = cv2.imread('media/de_blanco.jpg')
            elif race_detected == "indian":
                background = cv2.imread('media/Taj_Mahal.jpg')
            elif race_detected == "middle eastern":
                background = cv2.imread('media/mezquita.jpg')
            elif race_detected == "latino hispanic":
                background = cv2.imread('media/brazil.jpg')

            background = cv2.resize(background, (int(cap.get(3)), int(cap.get(4))))


            cancion_actual = canciones.get(race_detected, cancion_predeterminada)
            pygame.mixer.music.load(cancion_actual)
            pygame.mixer.music.play()
            # Esperar 5 segundos antes de cambiar el fondo nuevamente
            time.sleep(5)

# Crear un hilo para cambiar el fondo cada 5 segundos
fondo_thread = threading.Thread(target=cambiar_fondo)
fondo_thread.start()

while True:
    # Obtener el frame actual
    ret, frame = cap.read()

    # Para HS normalization
    B, G, R = cv2.split(frame)
    result = frame.copy()
    values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[1], FDet.EyeDetectors[1])

    if values is not None:
        face, eyes, shape = values

        # Dibujar el contenedor de la cara
        [x, y, w, h] = face
        if x > -1:
            cv2.rectangle(result, (x, y), (x + w, y + h), (255, 0, 0), 2)
            face_roi = frame[y:y+h, x:x+w]

            # Ejemplo de análisis de raza utilizando DeepFace
            obj = DeepFace.analyze(img_path=face_roi, enforce_detection=False, actions=['race'])
            race_detected = obj[0]["dominant_race"]

            # Superponer la región de la cara en el fondo
            background = cv2.resize(background, (int(cap.get(3)), int(cap.get(4))))
            result = background.copy()
            result[y:y+h, x:x+w] = cv2.addWeighted(face_roi, 0.9, result[y:y+h, x:x+w], 0.1, 0)

    # Mostrar el resultado
    cv2.imshow('Result', result)

    # Presionar 'Esc' para finalizar
    key = cv2.waitKey(40)
    if key == 27:  # Esc
        break

# Cerrar ventanas y liberar la cámara

pygame.mixer.music.stop()
pygame.mixer.quit()
cap.release()
cv2.destroyAllWindows()



Action: race: 100%|██████████| 1/1 [00:01<00:00,  1.18s/it]


Cambiando fondo


Action: race: 100%|██████████| 1/1 [00:01<00:00,  1.16s/it]


Cambiando fondo


Action: race: 100%|██████████| 1/1 [00:00<00:00,  1.74it/s]

Cambiando fondo



Action: race: 100%|██████████| 1/1 [00:01<00:00,  1.41s/it]


In [ ]:

cap = cv2.VideoCapture(0)
background = cv2.imread('media/africa.jpg')
background = cv2.resize(background, (int(cap.get(3)), int(cap.get(4))))

while True:
    ret, frame = cap.read()
    result = cv2.addWeighted(frame, 0.8, background, 0.7, 0)

    cv2.imshow('Result', result)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
folder = './media'

print(folder)

for file_name in os.listdir(folder):
    # Asume imágenes en formato png o jpg
    if file_name.endswith('.png') or file_name.endswith('.jpg'):
        # Procesa la imagen que asume hay cara, no fuerza la detección
        obj = DeepFace.analyze(img_path = os.path.join(folder, file_name), enforce_detection=False, actions =['race'])
        print(file_name)
        print(obj[0]['race'])
       
    